#COURSERA DATA SCIENCE CAPSTONE PROJECT

#Import Libraries

In [1]:
import pandas as pd # library for data analysis
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [70]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
#import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [13]:
pwd

'C:\\Users\\darry'

In [49]:
morris_zdata=pd.read_csv('C:\\Users\\darry\\Documents\\Coursera Data Science\\ZipLatLong2.csv')

In [50]:
morris_zdata

,Neighborhood,PostalCode,Latitude,Longitude
0,BaskingRidge,7920,40.6868,-74.5580
1,Bernardsville,7924,40.7296,-74.5815
2,Boonton,7005,40.9245,-74.4174
3,Brookside,7926,40.7954,-74.5663
4,BuddLake,7828,40.8903,-74.7454
5,Butler,7405,40.9886,-74.3822
6,Califon,7830,40.7383,-74.7922
7,CedarKnolls,7927,40.8236,-74.4555
8,Chatham,7928,40.7211,-74.4115
9,Chester,7930,40.7851,-74.6986


In [52]:
from pandas import DataFrame
#morris_df=pd.DataFrame(morris_zdata[0],columns=['Neighborhood','PostalCode','Longitude','Latitude'])
morris_df=pd.DataFrame(data=morris_zdata)
result=morris_df



In [53]:
#Analysis focused on Morris County NJ
# Generate maps to visualize neighborhoods and how they cluster together

In [54]:
import numpy as np # library to handle data in a vectorized manner
tor_latitude=40.7968
tor_longitude=-74.4815
neighborhoods=result
neighborhoods.head()

,Neighborhood,PostalCode,Latitude,Longitude
0,BaskingRidge,7920,40.6868,-74.5580
1,Bernardsville,7924,40.7296,-74.5815
2,Boonton,7005,40.9245,-74.4174
3,Brookside,7926,40.7954,-74.5663
4,BuddLake,7828,40.8903,-74.7454


In [55]:
# @hidden_cell
CLIENT_ID = 'KFP1T3LBOSSNVXOUC00PWGOGBJJWCIRQAKHEFTF1KZQMVESQ' # your Foursquare ID
CLIENT_SECRET = 'KST3GUTQF5TQ2IR1DOAMA2MDOOSL1G5P0YAQ42C4DGSU3HJ2' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [56]:
#Get nearby venues for each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'])            
            for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
morris_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

BaskingRidge
Bernardsville
Boonton
Brookside
BuddLake
Butler
Califon
CedarKnolls
Chatham
Chester
Denville
Dover
EastHanover
FarHills
Flanders
FlorhamPark
Gillette
GreenVillage
Hackettstown
Hibernia
Ironia
Kenvil
LakeHiawatha
LakeHopatcong
Landing
Ledgewood
LincolnPark
LongValley
Madison
Mendham
Millington
MineHill
Montville
MorrisPlains
Morristown
MountArlington
MountFreedom
MountTabor
MountainLakes
Netcong
NewProvidence
NewVernon
Newfoundland
OakRidge
Parsippany
Pequannock
PicatinnyArsenal
PineBrook
PomptonPlains
Randolph
Riverdale
Rockaway
SchooleysMountain
Stanhope
Stirling
Succasunna
Towaco
Wayne
Wharton
Whippany


In [58]:
# one hot encoding
morris_onehot = pd.get_dummies(morris_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
morris_onehot['Neighborhood'] = morris_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [morris_onehot.columns[-1]] + list(morris_onehot.columns[:-1])
morris_onehot = morris_onehot[fixed_columns]
morris_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Amphitheater,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,...,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Wine Shop,Yoga Studio
0,BaskingRidge,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bernardsville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bernardsville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Brookside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Brookside,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
morris_grouped = morris_onehot.groupby('Neighborhood').mean().reset_index()
morris_grouped 
num_top_venues = 5

In [60]:
for hood in morris_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = morris_grouped[morris_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BaskingRidge----
                 venue  freq
0   African Restaurant   1.0
1        Moving Target   0.0
2  Rental Car Location   0.0
3                  Pub   0.0
4          Post Office   0.0


----Bernardsville----
                        venue  freq
0  Construction & Landscaping   0.5
1                      Garden   0.5
2                    Pharmacy   0.0
3     New American Restaurant   0.0
4                   Nightclub   0.0


----Brookside----
                venue  freq
0      Baseball Field   0.5
1               Trail   0.5
2  African Restaurant   0.0
3           Pet Store   0.0
4         Music Venue   0.0


----Butler----
                      venue  freq
0                Smoke Shop  0.33
1  Mediterranean Restaurant  0.33
2          Greek Restaurant  0.33
3                  Pharmacy  0.00
4   New American Restaurant  0.00


----Califon----
                     venue  freq
0             Home Service   1.0
1       African Restaurant   0.0
2                Piano Bar   0.0
3  New

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
       
    return row_categories_sorted.index.values[0:num_top_venues]

In [65]:
num_top_venues=10
indicators = ['st', 'nd', 'rd']
for hood in morris_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = morris_grouped[morris_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BaskingRidge----
                 venue  freq
0   African Restaurant   1.0
1        Moving Target   0.0
2  Rental Car Location   0.0
3                  Pub   0.0
4          Post Office   0.0
5                 Pool   0.0
6           Playground   0.0
7          Pizza Place   0.0
8            Piano Bar   0.0
9             Pharmacy   0.0


----Bernardsville----
                        venue  freq
0  Construction & Landscaping   0.5
1                      Garden   0.5
2                    Pharmacy   0.0
3     New American Restaurant   0.0
4                   Nightclub   0.0
5           Other Repair Shop   0.0
6                        Park   0.0
7                   Pet Store   0.0
8          African Restaurant   0.0
9                 Music Venue   0.0


----Brookside----
                     venue  freq
0           Baseball Field   0.5
1                    Trail   0.5
2       African Restaurant   0.0
3                Pet Store   0.0
4              Music Venue   0.0
5  New American Restau

In [66]:
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [67]:
# create a new dataframe
neighborhoods_venues_sorted3 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted3['Neighborhood'] = morris_grouped['Neighborhood']

for ind in np.arange(morris_grouped.shape[0]):
    neighborhoods_venues_sorted3.iloc[ind, 1:] = return_most_common_venues(morris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted3.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BaskingRidge,African Restaurant,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Diner
1,Bernardsville,Construction & Landscaping,Garden,Yoga Studio,Diner,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Department Store
2,Brookside,Baseball Field,Trail,Yoga Studio,Dive Bar,Farmers Market,Farm,Electronics Store,Donut Shop,Doctor's Office,Diner
3,Butler,Smoke Shop,Greek Restaurant,Mediterranean Restaurant,Yoga Studio,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Diner,Department Store
4,Califon,Home Service,Yoga Studio,Diner,Farmers Market,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Department Store


In [ ]:
#Run K-Means to Cluster the Neighborhood

In [68]:
#Run k-means to cluster the neighborhood into 5 clusters
kclusters = 5
morris_grouped_clustering = morris_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(morris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted3.insert(0, 'Cluster Labels', kmeans.labels_)


morris_merged = neighborhoods# merge morris_grouped with morris_data to add latitude/longitude for each neighborhood
morris_merged = morris_merged.join(neighborhoods_venues_sorted3.set_index('Neighborhood'), on='Neighborhood')

morris_merged.head()


morris_merged.dropna(subset=['Cluster Labels'],inplace=True)
morris_merged

,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BaskingRidge,7920,40.6868,-74.5580,2.0,African Restaurant,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Diner
1,Bernardsville,7924,40.7296,-74.5815,2.0,Construction & Landscaping,Garden,Yoga Studio,Diner,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Department Store
3,Brookside,7926,40.7954,-74.5663,2.0,Baseball Field,Trail,Yoga Studio,Dive Bar,Farmers Market,Farm,Electronics Store,Donut Shop,Doctor's Office,Diner
5,Butler,7405,40.9886,-74.3822,2.0,Smoke Shop,Greek Restaurant,Mediterranean Restaurant,Yoga Studio,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Diner,Department Store
6,Califon,7830,40.7383,-74.7922,1.0,Home Service,Yoga Studio,Diner,Farmers Market,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Department Store
7,CedarKnolls,7927,40.8236,-74.4555,2.0,Construction & Landscaping,Sculpture Garden,Yoga Studio,Diner,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Deli / Bodega
8,Chatham,7928,40.7211,-74.4115,4.0,Farm,Yoga Studio,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store
9,Chester,7930,40.7851,-74.6986,2.0,Coffee Shop,Breakfast Spot,Ice Cream Shop,Sandwich Place,Mexican Restaurant,Cupcake Shop,Salon / Barbershop,Candy Store,Gift Shop,Playground
10,Denville,7834,40.8923,-74.4774,2.0,Italian Restaurant,Vegetarian / Vegan Restaurant,Pub,Pizza Place,Coffee Shop,Yoga Studio,Grocery Store,Seafood Restaurant,Sandwich Place,Salon / Barbershop
11,Dover,7801,40.8840,-74.5621,2.0,Mexican Restaurant,Italian Restaurant,Coffee Shop,Bank,Convenience Store,Deli / Bodega,Café,Shoe Store,Soccer Field,Spanish Restaurant


#DISPLAY CLUSTER RESULTS


In [73]:
#REVIEW CLUSTER RESULTS
morris_merged.loc[morris_merged['Cluster Labels'] == 0, morris_merged.columns[[1] + list(range(5, morris_merged.shape[1]))]]


,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,7931,Park,Speakeasy,Restaurant,Yoga Studio,Donut Shop,Doctor's Office,Dive Bar,Diner,Department Store,Deli / Bodega
20,7845,Italian Restaurant,American Restaurant,Park,Diner,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Yoga Studio
27,7853,Park,Yoga Studio,Fast Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Diner
58,7885,Park,Speakeasy,Spa,Department Store,Donut Shop,Doctor's Office,Dive Bar,Diner,Yoga Studio,Farm


In [74]:
morris_merged.loc[morris_merged['Cluster Labels'] == 1, morris_merged.columns[[1] + list(range(5, morris_merged.shape[1]))]]

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,7830,Home Service,Yoga Studio,Diner,Farmers Market,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Department Store
30,7946,Home Service,Yoga Studio,Diner,Farmers Market,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Department Store
38,7046,Home Service,Yoga Studio,Diner,Farmers Market,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Department Store
49,7869,Home Service,Piano Bar,Yoga Studio,Diner,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Deli / Bodega


In [75]:
morris_merged.loc[morris_merged['Cluster Labels'] == 2, morris_merged.columns[[1] + list(range(5, morris_merged.shape[1]))]]

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,7920,African Restaurant,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Diner
1,7924,Construction & Landscaping,Garden,Yoga Studio,Diner,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Department Store
3,7926,Baseball Field,Trail,Yoga Studio,Dive Bar,Farmers Market,Farm,Electronics Store,Donut Shop,Doctor's Office,Diner
5,7405,Smoke Shop,Greek Restaurant,Mediterranean Restaurant,Yoga Studio,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Diner,Department Store
7,7927,Construction & Landscaping,Sculpture Garden,Yoga Studio,Diner,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Deli / Bodega
9,7930,Coffee Shop,Breakfast Spot,Ice Cream Shop,Sandwich Place,Mexican Restaurant,Cupcake Shop,Salon / Barbershop,Candy Store,Gift Shop,Playground
10,7834,Italian Restaurant,Vegetarian / Vegan Restaurant,Pub,Pizza Place,Coffee Shop,Yoga Studio,Grocery Store,Seafood Restaurant,Sandwich Place,Salon / Barbershop
11,7801,Mexican Restaurant,Italian Restaurant,Coffee Shop,Bank,Convenience Store,Deli / Bodega,Café,Shoe Store,Soccer Field,Spanish Restaurant
12,7936,Flower Shop,Cafeteria,Yoga Studio,Diner,Farm,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Department Store
14,7836,Garden Center,Chinese Restaurant,Burger Joint,Yoga Studio,Electronics Store,Donut Shop,Doctor's Office,Dive Bar,Diner,Department Store


In [76]:
morris_merged.loc[morris_merged['Cluster Labels'] == 3, morris_merged.columns[[1] + list(range(5, morris_merged.shape[1]))]]

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,7874,Beer Garden,Field,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Diner,Dive Bar
55,7876,Beer Garden,Field,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Diner,Dive Bar


In [77]:
morris_merged.loc[morris_merged['Cluster Labels'] == 4, morris_merged.columns[[1] + list(range(5, morris_merged.shape[1]))]]

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,7928,Farm,Yoga Studio,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store
56,7082,Farm,Yoga Studio,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store
